## import Libraries

In [1]:
import nltk
import json
import warnings
warnings.filterwarnings("ignore")
nltk.download('punkt')
from nltk.chat.util import Chat, reflections
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem import WordNetLemmatizer
import pandas as pd
import random
from tensorflow.keras.layers import Dense,Conv2D,Dropout,BatchNormalization,Activation,Flatten
from tensorflow.keras.models import Sequential

[nltk_data] Downloading package punkt to C:\Users\Venkat
[nltk_data]     Karthi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Read the json 

In [2]:
with open('GL Bot.json') as file:
    corpus = json.load(file)

In [3]:
corpus

{'intents': [{'tag': 'Intro',
   'patterns': ['hi',
    'how are you',
    'You How',
    'is anyone there',
    'hello',
    'whats up',
    'What Doing',
    'hey',
    'yo',
    'listen',
    'hie',
    'Are you there',
    'please help me',
    'i am learner from',
    'i belong to',
    'aiml batch',
    'aifl batch',
    'i am from',
    'my pm is',
    'blended',
    'online',
    'i am from',
    'hey ya',
    'talking to you for first time'],
   'responses': ['Hello! how can i help you ?'],
   'context_set': ''},
  {'tag': 'Exit',
   'patterns': ['thank you',
    'thanks',
    'cya',
    'see you',
    'later',
    'see you later',
    'goodbye',
    'i am leaving',
    'have a Good day',
    'you helped me',
    'thanks a lot',
    'thanks a ton',
    'you are the best',
    'great help',
    'too good',
    'tata',
    'Bubye',
    'Thank you so much for your help',
    'Thank god You helped me',
    'you are a good learning buddy'],
   'responses': ['I hope I was able to as

## Convert into DF

In [4]:
final_df= pd.DataFrame(columns = ['patterns', 'tag', 'responses'])
for intent in corpus['intents']:
    df = pd.DataFrame(columns = ['patterns', 'tag'])
    df['patterns'] = intent['patterns']
    df['responses'] = intent['responses'][0]
    df['tag'] = intent['tag']
    final_df = final_df.append(df)

In [5]:
final_df

,patterns,tag,responses
0,hi,Intro,Hello! how can i help you ?
1,how are you,Intro,Hello! how can i help you ?
2,You How,Intro,Hello! how can i help you ?
3,is anyone there,Intro,Hello! how can i help you ?
4,hello,Intro,Hello! how can i help you ?
...,...,...,...
5,not good solution,Ticket,Tarnsferring the request to your PM
6,no help,Ticket,Tarnsferring the request to your PM
7,wasted my time,Ticket,Tarnsferring the request to your PM
8,useless bot,Ticket,Tarnsferring the request to your PM


## Train test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    final_df['patterns'].values, final_df['tag'].values, 
    test_size=0.15, random_state=0)
print('Training utterances: {}'.format(X_train.shape[0]))
print('Validation utterances: {}'.format(X_test.shape[0]))

Training utterances: 129
Validation utterances: 23


# First Approach

## Tfidf approach for NB

In [7]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)
X_train_vec, X_test_vec

(<129x167 sparse matrix of type '<class 'numpy.float64'>'
 	with 398 stored elements in Compressed Sparse Row format>,
 <23x167 sparse matrix of type '<class 'numpy.float64'>'
 	with 45 stored elements in Compressed Sparse Row format>)

## Build NB model

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
clf = MultinomialNB()
clf.fit(X_train_vec, y_train)
pred = clf.predict(X_test_vec)
print(accuracy_score(y_test, pred))

0.43478260869565216


In [9]:
X_test

array(['very diffult to understand neural nets', 'olympus',
       'i am not able to understand logistic regression', 'thank you',
       'artificial neural networks', 'great help', 'classification',
       'hey', 'forward propagation', 'whom to contact for olympus',
       'Bubye', 'hours of operation', 'Worst bot', 'cya',
       'artificial intelligence', 'wasted my time', 'relu',
       'lot of problems with olympus', 'batch size', 'you helped me',
       'screw you', 'i am not able to understand naive bayes',
       'working hours'], dtype=object)

In [10]:
y_test

array(['NN', 'Olympus', 'SL', 'Exit', 'NN', 'Exit', 'SL', 'Intro', 'NN',
       'Olympus', 'Exit', 'Bot', 'Profane', 'Exit', 'NN', 'Ticket', 'NN',
       'Olympus', 'NN', 'Exit', 'Profane', 'SL', 'Bot'], dtype=object)

In [11]:
pred

array(['NN', 'Olympus', 'SL', 'Exit', 'NN', 'Intro', 'SL', 'Intro', 'NN',
       'SL', 'SL', 'SL', 'SL', 'SL', 'SL', 'Intro', 'SL', 'Olympus',
       'Intro', 'Intro', 'Intro', 'SL', 'SL'], dtype='<U7')

## Chat utility function using NB

In [12]:
val = np.array(['Hi'])
val = vectorizer.transform(val)
print("BOT:",list(final_df[final_df['tag'] == clf.predict(val)[0]]['responses'])[0])

BOT: Hello! how can i help you ?


In [20]:
def chat():
    print('Hi This is Chitti!. How can I help you?')
    while True:
        inp = input("You:")    
        if inp.lower() == 'quit':
            break
        else:
            val = np.array([inp])
            val = vectorizer.transform(val)
            print("Chitti:",list(final_df[final_df['tag'] == clf.predict(val)[0]]['responses'])[0])
                

In [21]:
chat()

Hi This is Chitti!. How can I help you?
You:Hi
Chitti: Hello! how can i help you ?
You:whats up
Chitti: Hello! how can i help you ?
You:talking to you for first time
Chitti: Hello! how can i help you ?
You:thank you
Chitti: I hope I was able to assist you, Good Bye
You:you are the best
Chitti: I hope I was able to assist you, Good Bye
You:explain me how olympus works
Chitti: Link: Olympus wiki
You:lot of problem with olympus
Chitti: Link: Olympus wiki
You:i am not able to understand knn
Chitti: Link: Machine Learning wiki 
You:ML
Chitti: Link: Machine Learning wiki 
You:gradient boosting
Chitti: Link: Machine Learning wiki 
You:artificial intelligence
Chitti: Link: Machine Learning wiki 
You:unable to understand deep learning
Chitti: Link: Neural Nets wiki
You:Computer visin
Chitti: Link: Neural Nets wiki
You:name please
Chitti: Hello! how can i help you ?
You:what is your name
Chitti: I am your virtual learning assistant
You:screw you
Chitti: Hello! how can i help you ?
You:what the h

# Second Approach (Best One)

### Bag of words approach

In [22]:
W = []
L = []
doc_x = []
doc_y = []

for intent in corpus['intents']:
    for pattern in intent['patterns']:
        w_temp = nltk.word_tokenize(pattern)
        W.extend(w_temp)
        doc_x.append(w_temp)
        doc_y.append(intent['tag'])
    if intent['tag'] not in L:
        L.append(intent['tag'])
        
lemmatizer = WordNetLemmatizer()
W = [lemmatizer.lemmatize(w.lower()) for w in W]
W = sorted(list(set(W)))
L = sorted(L)

Train = []
Target = []
out_empty = [0 for _ in range(len(L))]

for x, doc in enumerate(doc_x):
    bag = []
    
    w_temp = [lemmatizer.lemmatize(w.lower()) for w in doc]
    
    for w in W:
        if w in w_temp:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[L.index(doc_y[x])] = 1
    
    Train.append(bag)
    Target.append(output_row)

## Neural Network

In [23]:
model = Sequential()
model.add(Dense(64, input_dim = len(Train[0]), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(Target[0]), activation = 'softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(Train, Target, epochs=100, batch_size=5, verbose= 1)

Epoch 1/100
31/31 [==============================] - 0s 2ms/step - loss: 2.1014 - accuracy: 0.0987
Epoch 2/100
31/31 [==============================] - 0s 2ms/step - loss: 2.0765 - accuracy: 0.1711
Epoch 3/100
31/31 [==============================] - 0s 2ms/step - loss: 2.0686 - accuracy: 0.1645
Epoch 4/100
31/31 [==============================] - 0s 2ms/step - loss: 1.9896 - accuracy: 0.2961
Epoch 5/100
31/31 [==============================] - 0s 2ms/step - loss: 1.9717 - accuracy: 0.2237
Epoch 6/100
31/31 [==============================] - 0s 2ms/step - loss: 1.9007 - accuracy: 0.3158
Epoch 7/100
31/31 [==============================] - 0s 2ms/step - loss: 1.8782 - accuracy: 0.3684
Epoch 8/100
31/31 [==============================] - 0s 2ms/step - loss: 1.7855 - accuracy: 0.3816
Epoch 9/100
31/31 [==============================] - 0s 2ms/step - loss: 1.7546 - accuracy: 0.3026
Epoch 10/100
31/31 [==============================] - 0s 2ms/step - loss: 1.6948 - accuracy: 0.4079
Epoch 11/

31/31 [==============================] - 0s 2ms/step - loss: 0.1589 - accuracy: 0.9474
Epoch 84/100
31/31 [==============================] - 0s 2ms/step - loss: 0.1366 - accuracy: 0.9737
Epoch 85/100
31/31 [==============================] - 0s 2ms/step - loss: 0.1718 - accuracy: 0.9737
Epoch 86/100
31/31 [==============================] - 0s 2ms/step - loss: 0.2055 - accuracy: 0.9408
Epoch 87/100
31/31 [==============================] - 0s 2ms/step - loss: 0.1795 - accuracy: 0.9474
Epoch 88/100
31/31 [==============================] - 0s 2ms/step - loss: 0.1799 - accuracy: 0.9408
Epoch 89/100
31/31 [==============================] - 0s 2ms/step - loss: 0.1662 - accuracy: 0.9474
Epoch 90/100
31/31 [==============================] - 0s 2ms/step - loss: 0.1793 - accuracy: 0.9539
Epoch 91/100
31/31 [==============================] - 0s 2ms/step - loss: 0.1691 - accuracy: 0.9408
Epoch 92/100
31/31 [==============================] - 0s 2ms/step - loss: 0.1366 - accuracy: 0.9868
Epoch 93/100


In [24]:
def bag_of_words(inp, W):
    lemmatizer = WordNetLemmatizer()
    inp_x = []
    Test = []
    w_temp = nltk.word_tokenize(inp)
    inp_x.append(w_temp)
    for x, doc in enumerate(inp_x):
        bag = []   
        w_temp = [lemmatizer.lemmatize(w.lower()) for w in doc]
        for w in W:
            if w in w_temp:
                bag.append(1)
            else:
                bag.append(0)

        Test.append(bag)
    return Test

In [25]:
def chat():
    print('Hi This is Chitti!. How can I help you?')
    while True:
        inp = input("You:")    
        if inp.lower() == 'quit':
            break
        results = model.predict(bag_of_words(inp, W))
        results_index = np.argmax(results)

        tag = L[results_index]

        for tg in corpus["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
                
        print(responses[0])

In [26]:
chat()

Hi This is Chitti!. How can I help you?
You:hello
Hello! how can i help you ?
You:how are you
Hello! how can i help you ?
You:Are you there
Hello! how can i help you ?
You:talking to you for first time
Hello! how can i help you ?
You:thank you
I hope I was able to assist you, Good Bye
You:Bubye
I hope I was able to assist you, Good Bye
You:explain me how olympus works
Link: Olympus wiki
You:lot of problems with olympus
Link: Olympus wiki
You:explain me how machine learning works
Link: Machine Learning wiki 
You:tuning
Link: Machine Learning wiki 
You:gradient
Link: Machine Learning wiki 
You:artificial intelligence
Link: Neural Nets wiki
You:Computer visin
Link: Neural Nets wiki
You:adam
Link: Neural Nets wiki
You:Say your details
I am your virtual learning assistant
You:what the hell
Please use respectful words
You:screw you
Please use respectful words
You:jerk
Please use respectful words
You:my problem is not solved
Tarnsferring the request to your PM
You:bad solution
Tarnsferring th

# Accuracy using Neural network is greater than that of Naive Bayes Classifier